In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress
from datetime import datetime

%matplotlib inline

# Impor the OpenWeatherMap API key
from config import api_key


In [ ]:
#Variables used in API call, oldest date is furthest back we are willing to go
oldest_date = "2014-01-31"
company_symbols = ["TAN",
                    "ENPH",
                    "SEDG",
                    "FSLR",
                    "HYDR",
                    "PLUG",
                    "BLDP",
                    "BE",
                    "FAN",
                    "TAC",
                    "TPIC",
                    "AMSC"]
time_function = "TIME_SERIES_MONTHLY"
base_url = "https://www.alphavantage.co/query?"

company_data = []

for symbol in company_symbols:
    finished_url = f"{base_url}function={time_function}&symbol={symbol}&apikey={api_key}"
    print(f"Processing {symbol}")

    result_data = requests.get(finished_url).json()
    print(result_data)

    time.sleep(12)

    for monthly in result_data["Monthly Time Series"]:
        monthly_date = time.strptime(monthly, "%Y-%m-%d")
        oldest = time.strptime(oldest_date, "%Y-%m-%d")
        if (monthly_date < oldest):
            continue
        stock_date = monthly
        stock_close = result_data["Monthly Time Series"][monthly]["4. close"]
        company_data.append({"Symbol":symbol,
                            "Date":stock_date,
                            "Closing Price":stock_close})

company_data_df = pd.DataFrame(company_data)
company_data_df.head()


In [ ]:
#Add a type column and lable to each stock
company_data_df["Type"] = ""
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "TAN", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "ENPH", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "SEDG", "Solar", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "FSLR", "Solar", company_data_df["Type"])

company_data_df["Type"] = np.where(company_data_df["Symbol"] == "HYDR", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "PLUG", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "BLDP", "Hydrogen", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "BE", "Hydrogen", company_data_df["Type"])

company_data_df["Type"] = np.where(company_data_df["Symbol"] == "FAN", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "TAC", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "TPIC", "Wind", company_data_df["Type"])
company_data_df["Type"] = np.where(company_data_df["Symbol"] == "AMSC", "Wind", company_data_df["Type"])

#Save the Data pulled to a CSV file
company_data_df.to_csv("data/company_data.csv", index_label="Stock_ID")

In [ ]:
#Read data from the saved CSV file and sort it by symbol and date
read_data_df = pd.read_csv("data/company_data.csv", index_col="Stock_ID")
ordered_data_df = read_data_df.sort_values(by=["Symbol", "Date"])
ordered_data_df

In [ ]:
print(ordered_data_df.columns)
# Define the stock symbols for each sector
solar_symbols = ["TAN", "ENPH", "SEDG", "FSLR"]
hydrogen_symbols = ["BLDP", "PLUG", "BE", "HYDR"]
wind_symbols = ["FAN", "TAC", "TPIC", "AMSC"]

#creating a data frame for each of the types
solar_symbols_df = ordered_data_df[ordered_data_df['Type'] == 'Solar']
solar_symbols_df.head()


In [ ]:
#creating the wind data frames
wind_symbols_df = ordered_data_df[ordered_data_df['Type'] == 'Wind']
wind_symbols_df.head()


In [ ]:
hydrogen_symbols_df = ordered_data_df[ordered_data_df['Type'] == 'Hydrogen']
hydrogen_symbols_df.head()

In [ ]:

# Calculate returns for each symbol
wind_symbols_df['Returns'] = wind_symbols_df.groupby('Symbol')['Closing Price'].pct_change()

# Calculate rolling volatility for each symbol (using a 20-day window)
window = 20
wind_symbols_df['Volatility'] = wind_symbols_df.groupby('Symbol')['Returns'].rolling(window=window).std().reset_index(0, drop=True) * (252 ** 0.5)  # Annualized

# Pivot the data to have one column per symbol
volatility_df = wind_symbols_df.pivot(index='Date', columns='Symbol', values='Volatility')

volatility_df.index = pd.to_datetime(volatility_df.index)

# Plotting
plt.figure(figsize=(12, 6))
for column in volatility_df.columns:
    plt.plot(volatility_df.index, volatility_df[column], label=column)

plt.title('Volatility of Wind Symbols Over Time')
plt.xlabel('Year')
plt.ylabel('Annualized Volatility')
plt.legend(title='Symbols', loc='upper right')
plt.ylim(bottom=0)

years=mdates.YearLocator()
years_fmt = mdates.DateFormatter('%Y')
ax=plt.gca()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)

plt.xlim(volatility_df.index.min() - pd.Timedelta(days=30),volatility_df.index.max() + pd.Timedelta(days=30))
plt.xticks(rotation=45)


plt.tight_layout()
plt.grid(True)
plt.show()

# Print the head of the volatility dataframe to check the data
print(volatility_df.head())

In [ ]:
# Calculate returns for each symbol
solar_symbols_df['Returns'] = solar_symbols_df.groupby('Symbol')['Closing Price'].pct_change()

# Calculate rolling volatility for each symbol (using a 20-day window)
window = 20
solar_symbols_df['Volatility'] = solar_symbols_df.groupby('Symbol')['Returns'].rolling(window=window).std().reset_index(0, drop=True) * (252 ** 0.5)  # Annualized

# Pivot the data to have one column per symbol
volatility_df = solar_symbols_df.pivot(index='Date', columns='Symbol', values='Volatility')

volatility_df.index = pd.to_datetime(volatility_df.index)

# Plotting
plt.figure(figsize=(12, 6))
for column in volatility_df.columns:
    plt.plot(volatility_df.index, volatility_df[column], label=column)

plt.title('Volatility of Solar Symbols Over Time')
plt.xlabel('Date')
plt.ylabel('Annualized Volatility')
plt.legend(title='Symbols', loc='upper right')
plt.ylim(bottom=0)

years=mdates.YearLocator()
years_fmt = mdates.DateFormatter('%Y')
ax=plt.gca()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)

plt.xlim(volatility_df.index.min() - pd.Timedelta(days=30),volatility_df.index.max() + pd.Timedelta(days=30))
plt.xticks(rotation=45)

plt.tight_layout()

plt.grid(True)

plt.show()


# Print the head of the volatility dataframe to check the data
print(volatility_df.head())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Calculate returns for each symbol
hydrogen_symbols_df['Returns'] = hydrogen_symbols_df.groupby('Symbol')['Closing Price'].pct_change()

# Calculate rolling volatility for each symbol (using a 20-day window)
#window = 20
hydrogen_symbols_df['Volatility'] = hydrogen_symbols_df.groupby('Symbol')['Returns'].std().reset_index(0, drop=True) * (252 ** 0.5)  # Annualized

# Pivot the data to have one column per symbol
volatility_df = solar_symbols_df.pivot(index='Date', columns='Symbol', values='Volatility')

volatility_df.index = pd.to_datetime(volatility_df.index)
# Plotting
plt.figure(figsize=(12, 6))
for column in volatility_df.columns:
    plt.plot(volatility_df.index, volatility_df[column], label=column)

plt.title('Volatility of Hydrogen Symbols Over Time')
plt.xlabel('Date')
plt.ylabel('Annualized Volatility')
plt.legend(title='Symbols', loc='upper right')
plt.ylim(bottom=0)

years=mdates.YearLocator()
years_fmt = mdates.DateFormatter('%Y')
ax=plt.gca()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)


plt.xlim(volatility_df.index.min() - pd.Timedelta(days=30),volatility_df.index.max() + pd.Timedelta(days=30))
plt.xticks(rotation=45) # Start y-axis at 0
plt.tight_layout()

plt.grid(True)
plt.show()


In [ ]:
# Ensure that the 'Date' column is in datetime format
ordered_data_df['Date'] = pd.to_datetime(ordered_data_df['Date'])


yearly_std = []

# Get unique company symbols
company_symbols = ordered_data_df['Symbol'].unique()

for symbol in company_symbols:
    # Get the starting year for the current symbol
    starting_year = ordered_data_df[ordered_data_df["Symbol"] == symbol]["Date"].dt.year.min()
    
    for year in range(starting_year, 2025):
        # Filter data for the current symbol and year
        yearly_data = ordered_data_df[(ordered_data_df["Symbol"] == symbol) & (ordered_data_df["Date"].dt.year == year)]
        
        # Calculate the standard deviation of the 'Closing Price'
        std = yearly_data["Closing Price"].std()
        
        # Append the result
        yearly_std.append({
            "Symbol": symbol,
            "Year": year,
            "Std for year": std,
            "Type": ordered_data_df.loc[ordered_data_df["Symbol"] == symbol, "Type"].iloc[0]
        })

# Create a DataFrame from the results
yearly_std_df = pd.DataFrame(yearly_std)

# Display the resulting DataFrame
yearly_std_df


In [ ]:
# Create a line plot for the yearly standard deviation
plt.figure(figsize=(12, 6))
for symbol in yearly_std_df['Symbol'].unique():
    symbol_data = yearly_std_df[yearly_std_df['Symbol'] == symbol]
    plt.plot(symbol_data['Year'], symbol_data['Std for year'], marker='o', label=symbol)

# Adding titles and labels
plt.title('Yearly Standard Deviation of Closing Prices by Company Symbol', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Standard Deviation of Closing Price', fontsize=14)
# Set x-ticks to show each year
plt.xticks(range(yearly_std_df['Year'].min(), 2025))  
plt.legend(title='Company Symbol', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()

# Show the plot
plt.show()




**Analysis:**
- This graph illustrates the closing price volatility for various companies from 2014 to 2024, revealing several notable trends. Between 2014 and 2018, most companies maintained relatively low and stable volatility levels. However, a significant shift occurred in 2019 and 2020, with numerous firms experiencing a marked increase in volatility, possibly due to economic disruptions such as the COVID-19 pandemic. 
- Among the companies represented, SEDG (likely SolarEdge) stands out as the most volatile, showing dramatic spikes in 2019, 2020, and 2022. While some companies like AMSC and BE have shown increased volatility in recent years, others have managed to maintain more consistent stability throughout the period. 
- Interestingly, as we approach 2024, the volatility for most companies appears to be on a downward trend, potentially signaling a move towards greater market stability. This comprehensive view of price volatility across different firms offers valuable insights into market dynamics and individual company performance over the past decade.

In [ ]:
plt.figure(figsize=(12, 6))
for symbol in yearly_std_df['Type'].unique():
    symbol_data = yearly_std_df[yearly_std_df['Type'] == symbol]
    plt.plot(symbol_data['Year'], symbol_data['Std for year'], marker='o', label=symbol)

# Adding titles and labels
plt.title('Yearly Standard Deviation of Closing Prices by Sector', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Standard Deviation of Closing Price', fontsize=14)
plt.xticks(range(yearly_std_df['Year'].min(), 2025))  # Set x-ticks to show each year
plt.legend(title='Sector', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.grid()

# Show the plot
plt.show()



**Analysis:**

The volatility of closing prices in many renewable energy industries is compared in this graph between 2014 and 2024. 

**Important realizations:**

- The most volatile industry overall is the solar sector, which had notable peaks in 2019 and 2020 as well as in 2022 and 2023.

- Over the course of the time, the wind and hydrogen sectors exhibit comparatively lower and more steady volatility.

- From 2018 to 2019, all sectors exhibit more volatility, perhaps as a result of changes in the business or world events.

- The tidal sector has shown to have the least amount of volatility overall, being steady over time.

- There is a convergence in sectoral volatility by 2024, indicating a potential stability or maturing of the industry.

Overall, the two graphs demonstrate that there were significant price fluctuations in the renewable energy sector between 2019 and 2023, with the largest fluctuations occurring in the solar sector and solar-related businesses. Even while volatility is considerably higher than it was in the first half of the decade, the data indicates a trend of stability by 2024.
